# Part 1: Extracting metadata from a rawpy object

In [1]:
from astropy.io import fits as fits
import rawpy

In [2]:
path = '../astrophotography_data/MilkyWayPrettyBoy/flats/raw_2/DSC06737.ARW'

raw = rawpy.imread(path)
print(raw)
raw.raw_image

array([[620, 812, 628, ..., 844, 644, 844],
       [816, 676, 816, ..., 676, 832, 676],
       [628, 816, 624, ..., 840, 644, 840],
       ...,
       [844, 680, 836, ..., 676, 824, 676],
       [636, 832, 628, ..., 836, 628, 836],
       [828, 692, 832, ..., 680, 836, 680]], dtype=uint16)

In [3]:
dir(raw)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'black_level_per_channel',
 'camera_whitebalance',
 'close',
 'color_desc',
 'color_matrix',
 'daylight_whitebalance',
 'dcraw_make_mem_image',
 'dcraw_make_mem_thumb',
 'dcraw_process',
 'extract_thumb',
 'num_colors',
 'open_buffer',
 'open_file',
 'postprocess',
 'raw_color',
 'raw_colors',
 'raw_colors_visible',
 'raw_image',
 'raw_image_visible',
 'raw_pattern',
 'raw_type',
 'raw_value',
 'raw_value_visible',
 'rgb_xyz_matrix',
 'sizes',
 'tone_curve',
 'unpack',
 'unpack_thumb']

In [4]:
raw.color_desc

b'RGBG'

In [5]:
raw.black_level_per_channel

[512, 512, 512, 512]

In [6]:
raw.raw_type

<RawType.Flat: 0>

In [7]:
raw.raw_pattern

array([[0, 1],
       [3, 2]], dtype=uint8)

In [8]:
raw.raw_colors[0:10,0:10]

array([[0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [3, 2, 3, 2, 3, 2, 3, 2, 3, 2],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [3, 2, 3, 2, 3, 2, 3, 2, 3, 2],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [3, 2, 3, 2, 3, 2, 3, 2, 3, 2],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [3, 2, 3, 2, 3, 2, 3, 2, 3, 2],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [3, 2, 3, 2, 3, 2, 3, 2, 3, 2]], dtype=uint8)

In [9]:
raw.rgb_xyz_matrix

array([[ 0.5838, -0.143 , -0.0246],
       [-0.3497,  1.1477,  0.2297],
       [-0.0748,  0.1885,  0.5778],
       [ 0.    ,  0.    ,  0.    ]], dtype=float32)

In [10]:
# Understanding processing
with rawpy.imread(path) as raw:
    
    # Actual raw pixel values are obtained after subtracting the
    # camera-created black level per channel. In the Sony A7, this
    # parameter is the same for all channels, and constant.
    # Just subtract it from everything.
    print(raw.raw_image[0:4,0:4] - 512) 

    # These settings preserve the raw pixel values into the output 
    # RGB array.
    rgb = raw.postprocess(gamma=(1,1), 
                          output_bps=16,
                          use_camera_wb=False,
                          use_auto_wb=False,
                          no_auto_bright=True, 
                          no_auto_scale=True,
                          output_color=rawpy.ColorSpace.raw)

    red = rgb[::,::,0]
    green = rgb[::,::,1]
    blue = rgb[::,::,2]

    print(red[0:2,0:2])
    print(green[0:2,0:2])
    print(blue[0:2,0:2])

[[108 300 116 296]
 [304 164 304 160]
 [116 304 112 296]
 [304 160 304 160]]
[[108 112]
 [112 113]]
[[302 300]
 [304 303]]
[[164 164]
 [164 164]]


# Part 2: Extracting EXIF metadata

In [11]:
import os,glob
import exifread

In [12]:
def print_exif(filename):
    f = open(filename, 'rb')
    tags = exifread.process_file(f)
    for tag in tags.keys():
#          if tag in ('EXIF ExposureTime', 'EXIF ISOSpeedRatings'):
         if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
            print("%s:      %s" % (tag, tags[tag]))

In [13]:
# filename = '../astrophotography_data/MilkyWayPrettyBoy/12800/dark/DSC03830.ARW'
filename = '../astrophotography_data/time_lapse_4/raw/DSC01396.ARW'
print_exif(filename)

Image SubfileType:      Reduced-resolution image
Image Compression:      JPEG (old-style)
Image ImageDescription:                                     
Image Make:      SONY
Image Model:      ILCE-7SM2
Image Orientation:      Horizontal (normal)
Image XResolution:      350
Image YResolution:      350
Image ResolutionUnit:      Pixels/Inch
Image Software:      ILCE-7SM2 v2.20
Image DateTime:      2019:06:27 01:34:42
Image Tag 0x014A:      138370
Image JPEGInterchangeFormat:      139426
Image JPEGInterchangeFormatLength:      323285
Image YCbCrPositioning:      Co-sited
Image ExifOffset:      436
Image PrintIM:      [80, 114, 105, 110, 116, 73, 77, 0, 48, 51, 48, 48, 0, 0, 3, 0, 2, 0, 1, 0, ... ]
Image Tag 0xC634:      [140, 173, 0, 0]
Thumbnail SubfileType:      Reduced-resolution image
Thumbnail Compression:      JPEG (old-style)
Thumbnail ImageDescription:                                     
Thumbnail Make:      SONY
Thumbnail Model:      ILCE-7SM2
Thumbnail Orientation:      Horizont

In [14]:
basepath = '../astrophotography_data/MilkyWayPrettyBoy/12800'

bias_path = os.path.join(basepath,'bias')
list_p = glob.glob(bias_path + '/*.ARW')
f = open(list_p[0], 'rb')
tags = exifread.process_file(f)
print(tags['EXIF ExposureTime'])

bias_path = os.path.join(basepath,'dark')
list_p = glob.glob(bias_path + '/*.ARW')
f = open(list_p[0], 'rb')
tags = exifread.process_file(f)
print(tags['EXIF ExposureTime'])

1/8000
16/5
